# Importing packages

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
import warnings
warnings.simplefilter("ignore")

# Train data

In [2]:
df=pd.read_csv('C:/Users/XolaniJ/Documents/EXPLORE ACADEMY_Data Science Course/ADVANCED CLASSIFICATION/NLP_Hackathon/train_set.csv')

# EDA

In [3]:
df.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [4]:
df.shape

(33000, 2)

In [5]:
df["lang_id"].value_counts()

tso    3000
sot    3000
nbl    3000
tsn    3000
afr    3000
xho    3000
eng    3000
zul    3000
ven    3000
ssw    3000
nso    3000
Name: lang_id, dtype: int64

<b>The training data has 33000 rows and two columns.Notably the classes that represents different languages have equal number (3000) of observations.</b>

In [6]:
df.isnull().sum()

lang_id    0
text       0
dtype: int64

<b>There are no missing values from the columns of the training data.</b>

# Test data

In [7]:
df_test=pd.read_csv('C:/Users/XolaniJ/Documents/EXPLORE ACADEMY_Data Science Course/ADVANCED CLASSIFICATION/NLP_Hackathon/test_set.csv')

In [8]:
df_test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [9]:
df_test.shape

(5682, 2)

<b>The test data has 5682 rows and two columns.</b>

In [10]:
df_test.isnull().sum()

index    0
text     0
dtype: int64

<b>There are no missing values from the test data.</b>

# Copy  datasets

In [11]:
train_lang = df.copy()
test_lang = df_test.copy()

# Combine datasets

In [12]:
DF = pd.concat([test_lang.assign(ind="test"), train_lang.assign(ind="train")])

In [13]:
DF.head()

,index,text,ind,lang_id
0,1.0,"Mmasepala, fa maemo a a kgethegileng a letlele...",test,NaN
1,2.0,Uzakwaziswa ngokufaneleko nakungafuneka eminye...,test,NaN
2,3.0,Tshivhumbeo tshi fana na ngano dza vhathu.,test,NaN
3,4.0,Kube inja nelikati betingevakala kutsi titsini...,test,NaN
4,5.0,Winste op buitelandse valuta.,test,NaN


# Convert a text column to lowercase

In [14]:
DF['text'] = DF['text'].str.lower()

# Define custormise stopwords

In [15]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [16]:
#english
stop1=['i','c','-','u','@', 'me', 'my', 'im','i m','myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", "i'm","i am",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [17]:
#setswana
stop2=['tsa me','tsa','re','rona','tsa gago','gago','rraalona','èna','lona','dira','se','lo','se batlang','batlang','ga bōnè','ga',
'bōnè','fa','tsamaya','go']

In [18]:
#Sot
stop3=['ke','nna','ka bonna','ka nna','ka','rona','ea rona','tsa rona','rona ka borona','uena','u joalo','joalo','u tla','tla','u ka','ea hau','uena',
'lona ka bolona','eena','hae','ka boeena','eona','ke eona','bona','tsa bona','tsa','ba','ena','ba le','le','e ne e','o ile a','hore','bakeng sa',
'e le','empa','eng','ba bang ba','kapa','ne','ea','ho','re','e leng','etsa','ba bona ba','haeba','joang','mong le e mong','etsang','mona','lokela ho',
'joalo','ke hobane’ng ha','theoha','lokela','re','hape','leha e le efe','ka mor’a hore','feela','mong le e mong','’na','rōna','tlas’a','haholo','ka ho','feela',
'haholo','pele','le eena o','tšoanang','o ile a','tsohle','mono','neng','hao','ka nako eo','e le','tsena','mo','o na le','hape','ka se khonang','ile a etsa',
'ha ho','ka ho fetisisa','ho feta','ba ba neng ba','bile','hona joale','lokela','pakeng tsa','se ke oa','ha a ntse a','tse seng kae','hang','ka bobeli','ho fihlela',
'ka nako ya','khahlanong le','etsa','ha e etse joalo','ha a so','ka a etsa joalo','mohlomong ha ho joalo','mohlomong','ha ho','ha ea lokela','ha e hloke','ha ho joalo',
 'ka mora','pele ho','nakong','kena','mabapi le','bakeng sa','nakong','ha a ntse a','ho fihlela',
 'kapa','tse ling','enngwe','hape','mmalwa',"ka 'ngoe",'ka bobeli','ho tswela pele',"tlas'a",'fetile','theoha','kantle','ka hare','tlase','nyoloha','ho tloha','ka tlase',
'che','tsa hao','hape','se ke oa','lokela ho ba le','ne a ke ke','entse','ha aa ka a etsa joalo' ,'ke ke','ha ea lokela','e ne e se','li ne li se joalo','ha ho joalo']

In [19]:
#Afr
stop4=['ek','my','ek self','myself','ons',"ons s'n",'onsself','jy','jou','julle','jy het','het','jy sal','sal','jy sou','sou','joune','jouself',
'julleself','hy','hom','syne','sy','homself','sy is','is','haar','haarne','haarself','dit','dit is','self','hulle','hul',"hulle s'n",
'hulself','wat','watter','wie','hierdie','daardie','dit sal','die','was','wees','gewees','wese','het','hê','gehad het','gehad','doen','het gedoen',
'en','maar','as','of','omdat','tot','totdat','terwyl','wyle','van','by','teen','deur','met','vir','oor','om', 'omheen', 'rondon','tussen','in','gedurende',
'voor','daarna','na','hierbo','bo','hieronder','onder','aan','tot','van','up','af','in','uit','buite','op','verby', 'weer','verder','dan','destyds','destyd',
'een keer','een maal','hier','daar','wanneer','waar','hoekom','hoe','almal','alle','enige','beide','elk','elke','min','meer','die meeste','meeste','ander',
'sommige','sulke','sodanig','geen','nee','ook nie','nie','ook','enigste','net','slegs','sleg','eie','besit','dieselfde','so','as','alte','baie','kan','blik',
'moenie','moes','nou','en','is nie','kon nie','kon','het nie','doen nie','doen','moes nie','mag nie','mag','moenie','moet','nodig het','het nie nodig nie','moet nie',
'was nie','was','sou nie','sou']

In [20]:
#IsiXhosa
stop5=['mna','wam','ngokwam','thina','yethu','eyethu','wena','nguwe','une','uza','ungathanda','eyakho','ngokwakho','ngokwenu','nina',
'yena','yakhe','ngokwakhe','eyakhe','yiyo','ngokwayo','bona','kubo','yabo','eyabo','ngeyabo','ngokwabo','intoni','ntoni','eyiphi','ngeyiphi',
'ngubani','ungubani','engubani','le','leya','lonto','izakuza','ezi','eziya','ezo','ndim','ngu','wayenjalo','unjalo','babenjalo','njalo',
'kuba','ukuba','ngokuba','sele','unayo','nayo','ebe','unayo','ndinayo','be','a','e','i','o','u','ukuba','yenza','uzoyenza','zoyenza','ukwenza',
'uyenzile','wenzile','ukwenzile','kwenzile','kwenza','ukwenza','kwaye','okunye','kunye','kodwa','ukuba','kuba','okanye','kanye','kuba','ukuba','ngoba',
'njenge','kude kube','kude','kube','ngeli xesha','ngeli','xesha','xana','xa','na','ye','nge','nje','ngokuba','malunga','ngokuchasene','chasene','bachasene','chasile','umchasile','aka','kaloku',
'phakathi','ungene','ukugqitha','gqitha','gqithile','gqithisa','gqithisile','ngexesha','ngaphambili','emva','mva','ngentla','ntla',
'ngezantsi','zantsi','ukuya','kuya','ukusuka','kusuka','suka','phezulu','ngaphezulu','nangaphezulu','na ngaphezulu','phantsi',
'ngaphantsi','nga phantsi','phakathi','ngaphandle','phandle','ivuliwe','vuliwe','icimile','cimile','ngaphaya','phaya',
'kwakhona','khona','kwa','kamva','emva koko','koko','emveni','apha','phaya','ya','pha','ngaphaya','ingathi','ngathi','nini','nithini',
 'uthini','thini','bathini','bathi','athi','besithi','esithi','sithi','phi','njani','unjani','onjani','banjani','konke','bonke','nayiphi na','nayiphi',
'zombini','nganye','zimbalwa','ezimbalwa','kaninzi','ninzi','nintsi','kanintsi','ezinintsi','inintsi','uninzi','isininzi','sininzi',
'enye','ezinye','enjalo','injalo','kunjalo','njalo','hayi','kuphela','yeyakho','nge yakho','yakho','nge','ngokufanayo','okufanayo','oku','ofanayo','fanayo',
'ke','kunokuba','nokuba','kuba','naye','ebe','kakhulu','khulu','ka','unako','ubunako','ebenako','ubu nako','ebe nako','ngaba',
'nje','andazi','andiyazi','ndiyayazi','ndiyazi','kufanele','okufaneleyo','kufanelekile','ngoku','ngokuya','kwakhona','kwaye',
'ngaba','akunjalo','kunjalo','ayikwazi','ayenzi','eyenza','enza','khange','ayisiyiyo','ayiseyiyo','íyiyo','yiyo','le','ngekhe',
'akufuneki','kufuneka','wayengekho','ebengekho','ngekho','ebekho','ukho','babengekho','bengekho','angekhe','soze','mhlawumbi','eli','elo','nelo','neli','nalu','eliya','eliyana']

In [21]:
#Zulu
stop6=["futhi","kahle","kakhulu","kanye","khona","kodwa","kungani","kusho","la","lakhe","lapho","mina","ngesikhathi","nje","phansi","phezulu","u","ukuba","ukuthi","ukuze","uma","wahamba","wakhe","wami","wase","wathi","yakhe","zakhe","zonke",
'mina','mina','wami','thina','yethu','okwethu','wena','uwena','uke','uzoba','ubungeke','ngeke','eyakho','wenu','nina','yena','okwakhe','unjalo','kanjalo','njalo','kunjalo','eyakhe','yena','kuyinto','uqobo','qobo','bona','bona','yona','zona',
 'yabo','ezabo','zabo','ngokwabo','lwabo','lwazo','yini','kuyini','ngani','manje','khona','lapho','okuyi','imuphi','muphi', 'ubani','bani','ngubani','lokhu','lokhuya','lesi','leli','ukuthi','lokho kuzoba','lokho','kuzoba','lezi','labo','lezo','lawo',
'ngi','yimi','kuyinto','kukhona','kwaba','zazikhona','zikhona','ikhona','kube','babe','bekulokhu','ukuba','unayo','banayo','kwadingeka','dingeka','kokuba','yenza','kuyakwenza','wenze','ukwenza','futhi','nafuthi','kanye','nakanye','kodwa','kepha','kanti',
'uma','nxa','noma','kumbe','kambe','konje','yini','manje','ngoba','ngalokho','lokho','njengoba','kuze kube','kuze kwabe','kuze','kwabe','ngenkathi','ngesikhathi','lapho','ye','ngase','duze','ngoba','nge','ne','na','mayelana','maqondana','ngokumelene','ncikene',
'ncikene','ebhekene','nqikene','phakathi','ngaphakathi','ungene','kungene','bangene','ngokusebenzisa','ngesikhathi','mhlana','mhla','kusa','ngaphambi','ngaphambili','phambili','ngemuva','emuva','kamuva','ngenhla','enhla','ngezansi','phansi','ngaphansi','ezansi',
'uku','kusuka','phezulu','ngaphezulu','enhla','phakathi','kuphakathi','kwaphakathi','phuma','phandle','vula','kuvaliwe','valiwe','cishe','cishile','cishiwe','ungene','phezu','seleyo','eqela','futhi','buye','phinda','waphinda','ngaphinda','ngokuqhubekayo','phambili',
'lapho-ke','lapho ke','lapho','ke','kanye','lapha','la','laphaya','nini','manxa','ngesikhathi','kuphi','phi','ngani','ngobani','ubani','ngani','yini','kanjani','njani','unjani','konke','kuze','noma yini','noma kuphi','kokubili','bobabili','yombili','ngamunye','emunye',
'sisinye','lowo nalowo','lowo','nalowo','okumbalwa','mbalwa','iningana','ncane','okuningi','kuningi','ngokweqileyo','ningi ngokwedlulele','ngokwedlulele','kakhulu','okunye','ezinye','nye','yingcosana','enjalo','njalo','ngenga','nje','cha','qha','ke','noma','hhayi','kuphela',
'okwakho','wami','lami','yami','ngeyami','nge yami','ngowami','ngelami','ngo wami','nge lami','ngokufanayo','ngo kufanayo','fanayo','ngakho-ke','ngakho ke','ngakho','ke','kune','kunoba','kunokuthi','kuna','futhi','ngokweqile','kakhulu','impela','ngempela','nge mpela','-noku','noku',
'ngakwazi','nga kwazi','kuthanda','isifiso','intando','nje','ungakwenzi','kufanele','ngokufanele','ngo kufanele','bekufanele','beku','manje','kabusha','akuzona','kuzona','zona','ayikwazanga','ayikwazanga','akazange','akusho','kusho','akunjalo','kunjalo','kanjalo','njalo','ngangingenalo',
'nganginalo','nginalo','akukaze','kaze','angikaze','akuyona','kuyona','kungenzeka','kungenzeki','akumele','kumele','akudingeki','isidingo','ukudinga','ukufuna','funa','dinga','akunjalo','akufanele','kwakungeyona','yona','babengekho','ngeke']

In [22]:
#Venda
stop7=['nne','inwi','ene','ula','vhala','fhala','tshila','tshone','ndi','la','ya','ni','vhone',' khou','na','nnyi','bva','ga',' fhi',
 'gai','do','lini','zwavhudi','livhuwa','khonani','pfi','vhona','vuwa','nga','si','kone','ha','a','naa','ee!','ee','hai!','hai','futhi','habe',
'tou','thi','u','ngafhi','ndo','ri','rine','vha','inwi','wa','la','tsha','zwa','dza','la','mini','ngani','hani','mini','-ifhio','ifhio',
'aa','ndaa','ro','pfa','khamusi','zwo','fara','duvha','li','divhi','humbela','vhe','vho','dzi','dze','dzo','dzu','yo','yi','yu','ye','yu']


In [23]:
#Southern Sotho
stop8=["a","ba","bane","bona","e","ea","eaba","empa","ena","ha","hae","hape","ho","hore","ka","ke","la","le","li","me","mo","moo","ne","o","oa","re","sa","se","tloha","tsa","tse",'eba','aowa','mohlômong','mohlômongwê','tše','tša','go']


In [24]:
#Xitsonga
stop9=['xewani','avuxeni','ahee','kunjhani','minjhani','ni','kona','ndzi','ndza','vonani','hi','wena','ra','huma','e','kwihi','mi',
'ku','u','va','ina','e-e','kumbe','xana','a','tivi','kombela','vula','sweswo','nakambe','xi','xana','wa','ke','mina','ndza','ri',
'ku']

In [25]:
stop = stop1+stop2+stop3+stop4+stop5+stop6+stop7+stop8+stop9

# Remove stopwords

In [26]:
DF['text'] = DF['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

# Removing emojis

In [28]:
filter_char = lambda c: ord(c) < 256
DF['text']= DF['text'].apply(lambda s: ''.join(filter(filter_char, s)))

# Removing punctuation

In [29]:
#Cleaning and removing punctuation
english_punctuations = string.punctuation
punctuation_list = english_punctuations
def cleaning_punctuation(text):
    translator =str.maketrans('','',punctuation_list)
    return text.translate(translator)
DF['text'] = DF['text'].apply(lambda x: cleaning_punctuation(x))

# Cleaning repeating characters

In [30]:
def cleaning_repeating_char(text):
    return re.sub(r'(.)\1+',r'\1',text)
DF['text'] = DF['text'].apply(lambda x: cleaning_repeating_char(x))

# Remove emails

In [31]:
def cleaning_email(data):
    return re.sub('@[^\s]+',' ',data)
DF['text'] = DF['text'].apply(lambda x: cleaning_email(x))

# Remove URL's

In [32]:
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)
DF['text'] = DF['text'].apply(lambda x: cleaning_URLs(x))

# Remove numbers

In [33]:
def cleaning_numbers(data):
    return re.sub('[0-9]+','',data)
DF['text'] = DF['text'].apply(lambda x: cleaning_numbers(x))

# Cross validation

In [34]:
tdif = TfidfVectorizer(strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',stop_words=None, ngram_range=(1,1))
X_DF = tdif.fit_transform(DF['text'])
X_test = X_DF[0:5682]
X_train = X_DF[5682:]
y =train_lang['lang_id']

In [37]:
models = []
models.append(('MNB', MultinomialNB()))
models.append(('RF', RandomForestClassifier()))
models.append(('LSVC', LinearSVC()))
models.append(('XGB', XGBClassifier()))

In [38]:
results = []
names = []
for name, model in models:
    cv_results = cross_val_score(model,X_train,y, cv=5,scoring = 'accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name,cv_results.mean(), cv_results.std())
    print(msg)

MNB: 0.997212 (0.000353)
RF: 0.961485 (0.002139)
LSVC: 0.996152 (0.000661)
XGB: 0.952545 (0.002464)


<b>The cross validation results suggests that the Multinomial NB has the average accuracy of 99.7%, and it is best performing model compare to other models.</b>

# Train a MultinomialNB()

In [39]:
model_mnb = MultinomialNB()
model_mnb.fit(X_train, y)

MultinomialNB()

# Prediction

In [40]:
mnb_pred = model_mnb.predict(X_test)
pred_lang=pd.DataFrame(mnb_pred,columns=['lang_id'])
XJ_subm =pd.DataFrame({'index':df_test.index + 1,'lang_id':pred_lang.lang_id})
XJ_subm.to_csv('C:/Users/XolaniJ/Documents/EXPLORE ACADEMY_Data Science Course/ADVANCED CLASSIFICATION/NLP_Hackathon/submit_1.csv',index=False)

# Train a MultinomialNB(alpha=0.05)

In [41]:
model_mnb = MultinomialNB(alpha=0.05)

In [42]:
model_mnb.fit(X_train, y)

MultinomialNB(alpha=0.05)

In [43]:
mnb_pred = model_mnb.predict(X_test)
pred_lang=pd.DataFrame(mnb_pred,columns=['lang_id'])
XJ_subm =pd.DataFrame({'index':df_test.index + 1,'lang_id':pred_lang.lang_id})
XJ_subm.to_csv('C:/Users/XolaniJ/Documents/EXPLORE ACADEMY_Data Science Course/ADVANCED CLASSIFICATION/NLP_Hackathon/submit_2.csv',index=False)

# MultinomialNB: Hyperparameter Tuning

In [44]:
params = model_mnb.get_params()
params

{'alpha': 0.05, 'class_prior': None, 'fit_prior': True}

In [45]:
params = {'alpha': np.arange(0.0, 1.05, 0.025)
         }

multinomial_nb_grid = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5)
multinomial_nb_grid.fit(X_train,y)

print('Best Parameters : ',multinomial_nb_grid.best_params_)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   34.6s finished


Best Parameters :  {'alpha': 0.025}


In [46]:
model_mnb = MultinomialNB(alpha=0.025)
model_mnb.fit(X_train, y)

MultinomialNB(alpha=0.025)

In [47]:
mnb_pred = model_mnb.predict(X_test)
pred_lang=pd.DataFrame(mnb_pred,columns=['lang_id'])
XJ_subm =pd.DataFrame({'index':df_test.index + 1,'lang_id':pred_lang.lang_id})
XJ_subm.to_csv('C:/Users/XolaniJ/Documents/EXPLORE ACADEMY_Data Science Course/ADVANCED CLASSIFICATION/NLP_Hackathon/submit_3.csv',index=False)